In [6]:
model_name = "skt/kogpt2-base-v2"
save_dir = model_name.replace("/", "__") + '__data-add'
critic_save_dir = "skt__kogpt2-base-v2"

### PPO 데이터셋 확인

In [7]:
import json
data_path_3_PPO = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict)) # 12000
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

### 필요한 라이브러리 추가

In [8]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

### 모델학습에 사용할 옵티마이저와 모델을 준비

In [11]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained=f"model/{save_dir}/output_1_SFT", lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=f"model/{critic_save_dir}/output_2_RM", lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        model_name, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [12]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

### PPO 학습에 쓸 데이터를 불러와 토크나이징

In [13]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

len(list_prompt)

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


12000

### PPO Trainer 선언

In [14]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

### PPO 학습 진행

In [15]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

actor.model.save_pretrained(f'./model/{save_dir}/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:18<00:00,  6.33s/it]


### RLHF가 적용된 custom chatgpt의 생성능력을 확인

In [16]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):저는 인공지능 AI이기 때문에 불고기용 고기의 종류, 판매처 및 위치, 부위에 따라 다를 수 있다고 생각합니다. 일반적으로는 일반적으로 한우는 일반적으로 한우를 사용하며, 다른 종류에서도 사용될 수 있습니다. 따라서 상황에 따라 적합한 메뉴 선택이 필요합니다. 에서는 일반적으로 불고기 양념으로 판매되며, 한우는 한우와 비슷한 고기와 함께 조리될 경우 됩니다. 불고기 한우의 주 재료는 돼지고기로 추정되며, 이는 보통 양고기 국물을 통해 제공됩니다. 에서는 양고기의 경우, 한우를 사용하며, 이는 일반적으로 한우를 사용한다고 됩니다.肉聖敎증명서를先注さらさら? 라고도 합니다.由束其秦皇堂恵眞敎さら\と士ん srubscurae: フーキミツフと関 さら.ジッキミンダジス 라고도 합니다.秦国境僧流上油できら小さら 에는 일반적으로 쇠고기에서 유래되는 것으로 보입니다.小宮

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):1950년으로 기록됩니다1950년. 으로 부통령직을 수행한. 대통령입니다. 에에 해당하는 정보로서. }  대통령. 대통령, }私後懷明 } 대통령 }, \ 들의 \'를 수행했습니다. } er'\'라는 를 ~. 으로 \'수행한 따라지지갑니다. } 으로잉클링할 } 私. \'" }\'하고 \' 으로\'를 \'",", 'token': 251} } } \'token': "어느 \'을 수행했습니다. "", 'token'    \'", \

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):시카고 오헤어 국제공항은 미국 일리노이 주 시카고에서 찾을 수 있습니다. Americano Birth of Taxiacru이션)은 시카고에서 출발하여 미국 일리노이 주 시카고 지역에 도달할 있습니다. 시카고 오헤어 국제공항은 미국 일리노이 주 시카고 시내에서 가까이, 항

#### 메모리 관리를 위해 캐시를 비우기

In [17]:
torch.cuda.empty_cache()